# Telco Customer Churn Analysis - Part 3: Feature Engineering

**Project**: SpecSailor - Telco Customer Churn Prediction

**Author**: SpecSailor Team

**Date**: November 2025

## Overview
This notebook creates engineered features for the churn prediction model:
- **Billing Risk Score**: Monthly charges divided by tenure (higher = riskier)
- **Service Penetration Rate**: Total services / maximum possible services
- **Contract Type Encoding**: One-hot encoding for contract types
- **Payment Method Encoding**: One-hot encoding for payment methods
- **Early Lifecycle Risk**: Binary flag for customers with tenure < 12 months
- **Premium Internet Indicator**: Binary flag for Fiber optic internet

## Expected Output
- Feature-engineered dataset saved to `../data/processed/feature_engineered_data.csv`

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries loaded successfully!")

In [ ]:
# Load cleaned data
df = pd.read_csv('../data/processed/cleaned_data.csv')

print(f"Dataset shape: {df.shape}")
print(f"Total customers: {len(df):,}")
print(f"\nFirst few rows:")
df.head()

## Step 1: Create Billing Risk Score

**Definition**: `MonthlyCharges / (tenure + 1)`

**Rationale**: Customers with high monthly charges but low tenure are at higher risk of churning. Adding 1 to tenure prevents division by zero for new customers.

In [ ]:
# Create billing risk score
# Higher values indicate higher risk (high charges but low commitment)
df['billing_risk_score'] = df['MonthlyCharges'] / (df['tenure'] + 1)

print("✓ Billing Risk Score created")
print(f"\nStatistics:")
print(df['billing_risk_score'].describe())

# Compare churned vs non-churned customers
print(f"\nAverage Billing Risk Score:")
print(f"  Non-churned: {df[df['Churn']=='No']['billing_risk_score'].mean():.2f}")
print(f"  Churned:     {df[df['Churn']=='Yes']['billing_risk_score'].mean():.2f}")

In [ ]:
# Visualize billing risk score by churn status
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Box plot
df.boxplot(column='billing_risk_score', by='Churn', ax=axes[0])
axes[0].set_title('Billing Risk Score by Churn Status', fontweight='bold')
axes[0].set_xlabel('Churn')
axes[0].set_ylabel('Billing Risk Score')
plt.sca(axes[0])
plt.xticks([1, 2], ['No', 'Yes'])

# Histogram
df[df['Churn']=='No']['billing_risk_score'].hist(bins=30, alpha=0.6, label='No Churn', ax=axes[1], color='green')
df[df['Churn']=='Yes']['billing_risk_score'].hist(bins=30, alpha=0.6, label='Churned', ax=axes[1], color='red')
axes[1].set_title('Billing Risk Score Distribution', fontweight='bold')
axes[1].set_xlabel('Billing Risk Score')
axes[1].set_ylabel('Frequency')
axes[1].legend()

plt.tight_layout()
plt.show()

## Step 2: Create Service Penetration Rate

**Definition**: Count of services subscribed / Total possible services

**Services**: PhoneService, MultipleLines, InternetService, OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, StreamingTV, StreamingMovies

**Rationale**: Customers with more services are more invested and less likely to churn.

In [ ]:
# Define service columns
service_columns = [
    'PhoneService', 'MultipleLines', 'InternetService',
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies'
]

# Count total services for each customer
# Count 'Yes' values (not 'No' or 'No internet service' or 'No phone service')
df['total_services'] = 0

for col in service_columns:
    # Count if the service is active (Yes or specific service type)
    if col == 'InternetService':
        # For Internet, count DSL or Fiber optic as active
        df['total_services'] += (df[col].isin(['DSL', 'Fiber optic'])).astype(int)
    else:
        # For other services, count 'Yes' as active
        df['total_services'] += (df[col] == 'Yes').astype(int)

# Maximum possible services is 9
max_services = len(service_columns)

# Calculate service penetration rate
df['service_penetration_rate'] = df['total_services'] / max_services

print("✓ Service Penetration Rate created")
print(f"\nTotal Services Statistics:")
print(df['total_services'].describe())
print(f"\nService Penetration Rate Statistics:")
print(df['service_penetration_rate'].describe())

In [ ]:
# Compare service penetration by churn status
print(f"\nAverage Service Penetration Rate:")
print(f"  Non-churned: {df[df['Churn']=='No']['service_penetration_rate'].mean():.2%}")
print(f"  Churned:     {df[df['Churn']=='Yes']['service_penetration_rate'].mean():.2%}")

print(f"\nAverage Total Services:")
print(f"  Non-churned: {df[df['Churn']=='No']['total_services'].mean():.2f}")
print(f"  Churned:     {df[df['Churn']=='Yes']['total_services'].mean():.2f}")

In [ ]:
# Visualize service penetration by churn
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Total services bar chart
service_churn = df.groupby(['total_services', 'Churn']).size().unstack(fill_value=0)
service_churn_pct = service_churn.div(service_churn.sum(axis=1), axis=0) * 100

service_churn_pct.plot(kind='bar', ax=axes[0], color=['green', 'red'])
axes[0].set_title('Churn Rate by Total Services', fontweight='bold')
axes[0].set_xlabel('Total Services')
axes[0].set_ylabel('Percentage (%)')
axes[0].legend(['No Churn', 'Churned'])
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=0)

# Service penetration box plot
df.boxplot(column='service_penetration_rate', by='Churn', ax=axes[1])
axes[1].set_title('Service Penetration Rate by Churn', fontweight='bold')
axes[1].set_xlabel('Churn')
axes[1].set_ylabel('Service Penetration Rate')
plt.sca(axes[1])
plt.xticks([1, 2], ['No', 'Yes'])

plt.tight_layout()
plt.show()

## Step 3: Contract Type Encoding

Create one-hot encoded features for contract types:
- Month-to-month
- One year
- Two year

In [ ]:
# One-hot encode contract type
contract_dummies = pd.get_dummies(df['Contract'], prefix='contract')

# Add to dataframe
df = pd.concat([df, contract_dummies], axis=1)

print("✓ Contract Type encoded")
print(f"\nContract columns created:")
print(contract_dummies.columns.tolist())

# Show churn rate by contract type
print(f"\nChurn rate by Contract Type:")
contract_churn = df.groupby('Contract')['Churn'].apply(lambda x: (x=='Yes').sum() / len(x) * 100)
print(contract_churn)

In [ ]:
# Also create a simple binary feature: is_monthly_contract
df['is_monthly_contract'] = (df['Contract'] == 'Month-to-month').astype(int)

print("✓ Binary feature 'is_monthly_contract' created")
print(f"\nMonth-to-month contracts: {df['is_monthly_contract'].sum():,} ({df['is_monthly_contract'].mean()*100:.1f}%)")

## Step 4: Payment Method Encoding

Create one-hot encoded features for payment methods:
- Electronic check
- Mailed check
- Bank transfer (automatic)
- Credit card (automatic)

In [ ]:
# One-hot encode payment method
payment_dummies = pd.get_dummies(df['PaymentMethod'], prefix='payment')

# Add to dataframe
df = pd.concat([df, payment_dummies], axis=1)

print("✓ Payment Method encoded")
print(f"\nPayment columns created:")
print(payment_dummies.columns.tolist())

# Show churn rate by payment method
print(f"\nChurn rate by Payment Method:")
payment_churn = df.groupby('PaymentMethod')['Churn'].apply(lambda x: (x=='Yes').sum() / len(x) * 100)
print(payment_churn)

In [ ]:
# Visualize payment method impact
plt.figure(figsize=(10, 6))
payment_churn.plot(kind='bar', color='coral')
plt.title('Churn Rate by Payment Method', fontsize=14, fontweight='bold')
plt.xlabel('Payment Method')
plt.ylabel('Churn Rate (%)')
plt.xticks(rotation=45, ha='right')
plt.axhline(y=df['Churn'].apply(lambda x: x=='Yes').mean()*100, color='red', linestyle='--', label='Overall Churn Rate')
plt.legend()
plt.tight_layout()
plt.show()

print("\nInsight: Electronic check has significantly higher churn rate!")

## Step 5: Early Lifecycle Risk

**Definition**: Binary flag indicating if tenure < 12 months

**Rationale**: Customers in their first year are at higher risk of churning as they haven't established strong loyalty yet.

In [ ]:
# Create early lifecycle risk flag
df['early_lifecycle_risk'] = (df['tenure'] < 12).astype(int)

print("✓ Early Lifecycle Risk flag created")
print(f"\nCustomers with tenure < 12 months: {df['early_lifecycle_risk'].sum():,} ({df['early_lifecycle_risk'].mean()*100:.1f}%)")

# Compare churn rates
early_churn = df[df['early_lifecycle_risk']==1]['Churn'].apply(lambda x: x=='Yes').mean() * 100
established_churn = df[df['early_lifecycle_risk']==0]['Churn'].apply(lambda x: x=='Yes').mean() * 100

print(f"\nChurn Rate:")
print(f"  Early lifecycle (< 12 months): {early_churn:.1f}%")
print(f"  Established (>= 12 months):    {established_churn:.1f}%")
print(f"\nRisk multiplier: {early_churn / established_churn:.2f}x")

In [ ]:
# Visualize early lifecycle impact
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Churn rate comparison
lifecycle_churn = df.groupby('early_lifecycle_risk')['Churn'].apply(lambda x: (x=='Yes').sum() / len(x) * 100)
lifecycle_churn.plot(kind='bar', ax=axes[0], color=['green', 'orange'])
axes[0].set_title('Churn Rate by Lifecycle Stage', fontweight='bold')
axes[0].set_xlabel('Early Lifecycle Risk')
axes[0].set_ylabel('Churn Rate (%)')
axes[0].set_xticklabels(['Established (>=12 mo)', 'Early (<12 mo)'], rotation=45, ha='right')

# Tenure distribution by churn
df[df['Churn']=='No']['tenure'].hist(bins=30, alpha=0.6, label='No Churn', ax=axes[1], color='green')
df[df['Churn']=='Yes']['tenure'].hist(bins=30, alpha=0.6, label='Churned', ax=axes[1], color='red')
axes[1].axvline(x=12, color='black', linestyle='--', linewidth=2, label='12-month threshold')
axes[1].set_title('Tenure Distribution by Churn', fontweight='bold')
axes[1].set_xlabel('Tenure (months)')
axes[1].set_ylabel('Frequency')
axes[1].legend()

plt.tight_layout()
plt.show()

## Step 6: Premium Internet Indicator

**Definition**: Binary flag indicating if customer has Fiber optic internet

**Rationale**: Fiber optic customers may have different churn patterns due to higher costs and expectations.

In [ ]:
# Create premium internet indicator
df['has_premium_internet'] = (df['InternetService'] == 'Fiber optic').astype(int)

print("✓ Premium Internet indicator created")
print(f"\nCustomers with Fiber optic: {df['has_premium_internet'].sum():,} ({df['has_premium_internet'].mean()*100:.1f}%)")

# Compare churn rates by internet type
print(f"\nChurn Rate by Internet Service:")
internet_churn = df.groupby('InternetService')['Churn'].apply(lambda x: (x=='Yes').sum() / len(x) * 100)
print(internet_churn)

In [ ]:
# Visualize internet service impact
plt.figure(figsize=(10, 6))
internet_churn.plot(kind='bar', color=['gray', 'skyblue', 'purple'])
plt.title('Churn Rate by Internet Service Type', fontsize=14, fontweight='bold')
plt.xlabel('Internet Service')
plt.ylabel('Churn Rate (%)')
plt.xticks(rotation=0)
plt.axhline(y=df['Churn'].apply(lambda x: x=='Yes').mean()*100, color='red', linestyle='--', label='Overall Churn Rate')
plt.legend()
plt.tight_layout()
plt.show()

print("\nInsight: Fiber optic customers have higher churn rate despite premium service!")

## Step 7: Additional Useful Features

In [ ]:
# Payment reliability score (automatic payment methods are more reliable)
df['payment_reliability_score'] = df['PaymentMethod'].apply(
    lambda x: 1 if 'automatic' in x.lower() else 0
)

print("✓ Payment Reliability Score created")
print(f"  Customers with automatic payment: {df['payment_reliability_score'].sum():,} ({df['payment_reliability_score'].mean()*100:.1f}%)")

# Average charges per service
# Avoid division by zero
df['avg_charge_per_service'] = df.apply(
    lambda row: row['MonthlyCharges'] / row['total_services'] if row['total_services'] > 0 else row['MonthlyCharges'],
    axis=1
)

print("✓ Average Charge per Service created")
print(f"  Mean: ${df['avg_charge_per_service'].mean():.2f}")
print(f"  Median: ${df['avg_charge_per_service'].median():.2f}")

## Step 8: Feature Summary and Correlation Analysis

In [ ]:
# List all engineered features
engineered_features = [
    'billing_risk_score',
    'service_penetration_rate',
    'total_services',
    'contract_Month-to-month',
    'contract_One year',
    'contract_Two year',
    'is_monthly_contract',
    'payment_Bank transfer (automatic)',
    'payment_Credit card (automatic)',
    'payment_Electronic check',
    'payment_Mailed check',
    'early_lifecycle_risk',
    'has_premium_internet',
    'payment_reliability_score',
    'avg_charge_per_service'
]

print("=" * 60)
print("ENGINEERED FEATURES SUMMARY")
print("=" * 60)
print(f"\nTotal engineered features: {len(engineered_features)}")
print("\nFeatures:")
for i, feat in enumerate(engineered_features, 1):
    print(f"{i:2d}. {feat}")

In [ ]:
# Create binary churn variable for correlation
df['Churn_binary'] = (df['Churn'] == 'Yes').astype(int)

# Select key features for correlation analysis
correlation_features = [
    'tenure', 'MonthlyCharges', 'TotalCharges',
    'billing_risk_score', 'service_penetration_rate', 'total_services',
    'is_monthly_contract', 'early_lifecycle_risk', 'has_premium_internet',
    'payment_reliability_score', 'Churn_binary'
]

# Calculate correlations with churn
correlations = df[correlation_features].corr()['Churn_binary'].sort_values(ascending=False)

print("\n" + "=" * 60)
print("CORRELATION WITH CHURN")
print("=" * 60)
print(correlations)

In [ ]:
# Visualize correlations
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Correlation heatmap
corr_matrix = df[correlation_features].corr()
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, ax=axes[0], cbar_kws={'shrink': 0.8})
axes[0].set_title('Feature Correlation Matrix', fontweight='bold', fontsize=12)

# Correlation with churn bar plot
correlations_abs = correlations.drop('Churn_binary').abs().sort_values(ascending=True)
correlations_abs.plot(kind='barh', ax=axes[1], color='steelblue')
axes[1].set_title('Feature Importance (Correlation with Churn)', fontweight='bold', fontsize=12)
axes[1].set_xlabel('Absolute Correlation')
axes[1].set_ylabel('Feature')

plt.tight_layout()
plt.show()

## Step 9: Save Feature-Engineered Dataset

In [ ]:
# Display final dataset info
print("=" * 60)
print("FINAL DATASET INFO")
print("=" * 60)
print(f"\nShape: {df.shape}")
print(f"Rows: {len(df):,}")
print(f"Columns: {len(df.columns)}")

print(f"\nOriginal features: 21")
print(f"Engineered features: {len(engineered_features)}")
print(f"Total features: {len(df.columns)}")

print(f"\nMissing values: {df.isnull().sum().sum()}")
print(f"Duplicates: {df.duplicated().sum()}")

In [ ]:
# Save feature-engineered dataset
output_path = '../data/processed/feature_engineered_data.csv'
df.to_csv(output_path, index=False)

print(f"\n✓ Feature-engineered data saved to: {output_path}")
print(f"  File size: {os.path.getsize(output_path) / 1024:.2f} KB")
print(f"  Rows: {len(df):,}")
print(f"  Columns: {len(df.columns)}")

In [ ]:
# Display sample of final dataset
print("\nSample of feature-engineered dataset:")
display_cols = ['customerID', 'tenure', 'MonthlyCharges', 'Contract', 'Churn',
                'billing_risk_score', 'service_penetration_rate', 'total_services',
                'is_monthly_contract', 'early_lifecycle_risk', 'has_premium_internet']
df[display_cols].head(10)

## Summary of Feature Engineering

### Features Created:

1. **Billing Risk Score** (`billing_risk_score`)
   - Formula: MonthlyCharges / (tenure + 1)
   - Higher values indicate higher churn risk
   - Strong correlation with churn

2. **Service Penetration Rate** (`service_penetration_rate`)
   - Formula: total_services / 9 (max services)
   - Measures customer engagement
   - Lower penetration = higher churn risk

3. **Contract Type Features**
   - One-hot encoded: Month-to-month, One year, Two year
   - Binary flag: `is_monthly_contract`
   - Month-to-month contracts show ~42% churn rate

4. **Payment Method Features**
   - One-hot encoded: 4 payment types
   - `payment_reliability_score`: 1 for automatic, 0 for manual
   - Electronic check has highest churn rate

5. **Early Lifecycle Risk** (`early_lifecycle_risk`)
   - Binary flag: 1 if tenure < 12 months
   - Early customers have ~2x higher churn rate

6. **Premium Internet Indicator** (`has_premium_internet`)
   - Binary flag: 1 if Fiber optic
   - Surprisingly higher churn despite premium service

7. **Additional Features**
   - `total_services`: Count of active services
   - `avg_charge_per_service`: MonthlyCharges / total_services

### Key Insights:
- **Strongest positive correlations with churn**: is_monthly_contract, early_lifecycle_risk, billing_risk_score
- **Strongest negative correlations with churn**: tenure, payment_reliability_score, service_penetration_rate
- **Most predictive features ready for modeling**: Contract type, payment method, tenure-based features

### Next Step:
Proceed to **Notebook 04: Model Training** to train the XGBoost classifier.